In [1]:
import pandas as pd
from Bio import Entrez

Entrez.email = 'kaedeito@student.ubc.ca'
Entrez.tool = 'Chlamydomonas-metal-toxicity'

In [2]:
references_df = pd.read_excel('../references/references.xlsx', sheet_name='list')
references_df.head()

,Checked?,included?,paper downloaded?,BioProject,Gene Expression Omnibus,DOI,description,"Notes (searched using ""Chlamydomonas"" ""metal"" ""toxicity"" ""SRA"")"
0,True,False,NaN,NaN,NaN,10.1128/AEM.00998-13,Transcriptome Sequencing (RNA-seq) Analysis of...,"Dana F. Simon, Rute F. Domingos. No SRAs avail..."
1,True,False,NaN,PRJNA799651,NaN,NaN,Insights from comparative transcriptome analys...,don’t include
2,True,False,0.0,SRP040767,NaN,10.1371/journal.pone.0107092,NaN,"control, AgNO3 and AgNP, metatranscriptome, th..."
3,True,True,1.0,PRJNA608616,NaN,10.1093/plphys/kiab375,Long-term acclimation to cadmium exposure reve...,"combo of control (Ctrl), short-term (ST), and ..."
4,True,True,1.0,PRJNA735693,GSE176268,10.1016/j.envpol.2021.117594,Rare earth elements show antagonistic interact...,"(5 replicates for each treatment: Ce, Tm, Y, M..."


In [3]:
bioprojects_df = references_df[references_df['included?'] == True]
bioprojects_list = bioprojects_df['BioProject'].tolist()
bioprojects_df.head()

,Checked?,included?,paper downloaded?,BioProject,Gene Expression Omnibus,DOI,description,"Notes (searched using ""Chlamydomonas"" ""metal"" ""toxicity"" ""SRA"")"
3,True,True,1.0,PRJNA608616,NaN,10.1093/plphys/kiab375,Long-term acclimation to cadmium exposure reve...,"combo of control (Ctrl), short-term (ST), and ..."
4,True,True,1.0,PRJNA735693,GSE176268,10.1016/j.envpol.2021.117594,Rare earth elements show antagonistic interact...,"(5 replicates for each treatment: Ce, Tm, Y, M..."
5,True,True,1.0,PRJNA335844,GSE84995,10.1016/j.watres.2017.05.020,Transcriptomic approach for assessment of the ...,"Valcea is control, Babeni is experiment"
6,True,True,1.0,PRJNA394256,NaN,10.1016/j.aquatox.2018.04.020,Comparison of genetic differences and transcri...,"strain is the same, methodology is the same, s..."
7,True,True,1.0,PRJNA576974,NaN,10.3389/fmicb.2020.01443,NaN,Control 1: SRR10269729 \nControl 2: SRR102697...


In [13]:
# search for SRAs associated with each BioProject
from Bio.Entrez.Parser import DictionaryElement
import os

def get_sra_list(override: bool = False):
    sra_details_csv = '../references/sra_list.csv'
    # if the folder is not empty
    directory = '../sra_details'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # if the folder is not empty
    list_dir = os.listdir(directory)

    if (len(list_dir) == 0 ) or (override):
        print('SRA details not downloaded')
        sra_list = []
        for bioproject in bioprojects_list:
            print(f'Searching for SRA associated with {bioproject}')
            handle = Entrez.esearch(db='sra', term=bioproject, retmode='xml', rettype='uilist')
            record = Entrez.read(handle, validate=False)
            if isinstance(record, DictionaryElement):
                id_list: list[str] | None = record['IdList']
                if (id_list is None) or len(id_list) == 0:
                    handle.close()
                    raise Exception(f'No SRA found for {bioproject}')
                for id in id_list:
                    entry = { 'BioProject': bioproject, 'SRA_ID': id }
                    sra_list.append(entry)
                handle.close()
            else:
                handle.close()
                raise Exception(f'No SRA found for {bioproject}')

        sra_df = pd.DataFrame(sra_list)
        sra_df.to_csv(sra_details_csv, index=False)
        sra_df['SRA_ID'].to_csv('../references/sra_id_list.txt', header=None, index=None) # type: ignore
        return sra_list, sra_df
    else:
        # if the folder is not empty
        print('SRA details already downloaded')
        sra_df = pd.read_csv(sra_details_csv)
        sra_list: list[dict] = sra_df.to_dict('records')
        return sra_list, sra_df



sra_list, sra_df = get_sra_list()
sra_df['SRA_ID']=sra_df['SRA_ID'].astype(int)


SRA details already downloaded


In [5]:
import os

def get_sra_details(sra_list: list[dict[str, str]], override: bool = False):
  directory = '../sra_details'
  if not os.path.exists(directory):
    os.makedirs(directory)
  # if the folder is not empty
  list_dir = os.listdir(directory)
  if override or len(list_dir) == 0:
    xml_paths: list[str] = []
    for sra_id_dict in sra_list:
      sra_id = sra_id_dict.get('SRA_ID')
      print(f'Downloading {sra_id}...')
      handle = Entrez.efetch(db='sra', id=sra_id, retmode='xml', rettype='xml')
      xml_path = os.path.realpath(f'../sra_details/{sra_id}.xml')
      with open(xml_path, 'wb') as f:
        f.write(handle.read()) # type: ignore
      handle.close()
      xml_paths.append(xml_path)
    return xml_paths

  else:
      # if the folder is not empty
      # filter out .gitkeep
      list_dir = list(filter(lambda x: x != '.gitkeep', list_dir))
      # give full path
      list_dir = list(map(lambda x: os.path.realpath(f'{directory}/{x}'), list_dir))
      print('SRA details already downloaded')
      return list_dir


In [6]:
xml_paths = get_sra_details(sra_list)
print(xml_paths)

SRA details already downloaded
['E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287607.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287608.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287609.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287610.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287611.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287612.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287613.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287614.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287615.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287616.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287617.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\10287618.xml', 'E:\\Kaede\\Documents\\GitHub\\MICB405_FINAL\\sra_details\\14749612.xml', 'E:\\K

In [66]:
from xml.etree.ElementTree import Element
import xml.etree.ElementTree as ET

def get_design_details(design_ele: Element | None):
  design = {}

  if design_ele is None:
    return design
  library = design_ele.find('LIBRARY_DESCRIPTOR')
  if library is not None:
    library_source = library.find('LIBRARY_SOURCE')
    if library_source is not None:
      design['library_source'] = library_source.text
    library_strategy = library.find('LIBRARY_STRATEGY')
    if library_strategy is not None:
      design['library_strategy'] = library_strategy.text
    library_selection = library.find('LIBRARY_SELECTION')
    if library_selection is not None:
      design['library_selection'] = library_selection.text
    library_name = library.find('LIBRARY_NAME')
    if library_name is not None:
      design['library_name'] = library_name.text
  return design


def parse_xml_files(xml_paths: list[str]):
  entries: list[dict[str, str]] = []
  for xml_path in xml_paths:
    with open(xml_path, 'r') as f:
      os.path.basename(xml_path)
      sra_id = os.path.basename(xml_path).split('.')[0]
      tree = ET.parse(f)
      root = tree.getroot()

      exp_pckgs = root.findall('./EXPERIMENT_PACKAGE')
      if not exp_pckgs:
        continue

      for exp_pckg in exp_pckgs:

        exp = exp_pckg.find('EXPERIMENT')
        if exp is None:
          continue

        exp_id = exp.get('accession')
        title_ele = exp.find('TITLE')

        design_dict = get_design_details(exp.find('DESIGN'))

        run_set = exp_pckg.find('RUN_SET')
        if run_set is None:
          continue
        runs = run_set.findall('RUN')
        if not runs:
          continue

        for run in runs:
          stats = run.find('Statistics')
          nreads = stats.get('nreads') if stats is not None else None

          run_dict = {
            'SRX_ID': exp_id,
            'SRR_ID': run.get('accession'),
            'N_RUNS': int(nreads) if nreads is not None else None,
            'SRA_ID': sra_id,
            'title': title_ele.text if title_ele is not None else None,
          }
          entry = run_dict | design_dict
          entries.append(entry)
  return entries


In [67]:
entries = parse_xml_files(xml_paths)
entries_df = pd.DataFrame(entries)
entries_df['SRA_ID']=entries_df['SRA_ID'].astype(int)
entries_df.head()

,SRX_ID,SRR_ID,N_RUNS,SRA_ID,title,library_source,library_strategy,library_selection,library_name
0,SRX7859762,SRR11249513,2,10287607,RNA-Seq of Chlamydomonas cells: ctrl conditions,TRANSCRIPTOMIC,RNA-Seq,RANDOM,HMIBPS13
1,SRX7859763,SRR11249512,2,10287608,RNA-Seq of Chlamydomonas cells: ctrl conditions,TRANSCRIPTOMIC,RNA-Seq,RANDOM,HMIBPS14
2,SRX7859764,SRR11249511,2,10287609,RNA-Seq of Chlamydomonas cells: 50 microM Cd 6...,TRANSCRIPTOMIC,RNA-Seq,RANDOM,HMIBPS23
3,SRX7859765,SRR11249510,2,10287610,RNA-Seq of Chlamydomonas cells: 50 microM Cd 6...,TRANSCRIPTOMIC,RNA-Seq,RANDOM,HMIBPS24
4,SRX7859766,SRR11249509,2,10287611,RNA-Seq of Chlamydomonas cells: ctrl conditions,TRANSCRIPTOMIC,RNA-Seq,RANDOM,HMIBPS15


In [56]:
# iterate through the rows of the dataframe all_df
from io import StringIO
import requests

def get_ftp_url(SRR_IDS: list[str]):
  ena_col = ['run_accession', 'fastq_ftp', 'fastq_md5', 'fastq_bytes']
  ena_df = pd.DataFrame(columns=ena_col)

  for srr_id in SRR_IDS:
    ena_url = f"http://www.ebi.ac.uk/ena/portal/api/filereport?accession={srr_id}&result=read_run&fields=run_accession,fastq_ftp,fastq_md5,fastq_bytes"
    response = requests.get(ena_url)
    res_df = pd.read_csv(StringIO(response.text), sep='\t')[['run_accession', 'fastq_ftp', 'fastq_md5', 'fastq_bytes']]
    ena_df = pd.concat([ena_df, res_df])
  return ena_df

ftp_df = get_ftp_url(entries_df['SRR_ID'].tolist())

In [57]:
ftp_df.to_csv('../references/ftp_list.csv', index=False)

In [60]:
# split the col 'fastq_ftp' into two cols by ";", create a new row for each col

ftp_df_copy = ftp_df.copy()
ftp_df_copy["fastq_ftp"]=ftp_df_copy["fastq_ftp"].str.split(";")
ftp_df_copy = ftp_df_copy.explode("fastq_ftp").reset_index(drop=True)

ftp_df_copy

,run_accession,fastq_ftp,fastq_md5,fastq_bytes
0,SRR11249513,ftp.sra.ebi.ac.uk/vol1/fastq/SRR112/013/SRR112...,cb13bce5cbfd691d6f0769e10f07f0b6,1080038168
1,SRR11249512,ftp.sra.ebi.ac.uk/vol1/fastq/SRR112/012/SRR112...,88a088e0447d6ab9420c13eeda5dc216,1046541245
2,SRR11249511,ftp.sra.ebi.ac.uk/vol1/fastq/SRR112/011/SRR112...,f9ee6691aeb4df884f8c75a58d15ade8,1066664376
3,SRR11249510,ftp.sra.ebi.ac.uk/vol1/fastq/SRR112/010/SRR112...,69f5e02ac0d70d73fb1019b812b55c94,1114259453
4,SRR11249509,ftp.sra.ebi.ac.uk/vol1/fastq/SRR112/009/SRR112...,c0ea9f54ec357f6e754080d183b0575a,963887385
...,...,...,...,...
83,SRR10269729,ftp.sra.ebi.ac.uk/vol1/fastq/SRR102/029/SRR102...,0463bb2f0eda70b2dfbd70ba86a06e59;7b01790d5838a...,2172817555;2215727670
84,SRR10269728,ftp.sra.ebi.ac.uk/vol1/fastq/SRR102/028/SRR102...,a110c6c4348a3ec01fa20216da5b89bf;42aec9e0f1087...,1957065715;2006644859
85,SRR10269728,ftp.sra.ebi.ac.uk/vol1/fastq/SRR102/028/SRR102...,a110c6c4348a3ec01fa20216da5b89bf;42aec9e0f1087...,1957065715;2006644859
86,SRR10269727,ftp.sra.ebi.ac.uk/vol1/fastq/SRR102/027/SRR102...,075c7a3df2fbfadfdf86667e9a9d103c;74ae52000e9e6...,2057257939;2110346713


In [68]:
all_df = pd.merge(sra_df, entries_df, on='SRA_ID')
all_df = pd.merge(all_df, bioprojects_df, on='BioProject')
all_df = pd.merge(all_df, ftp_df_copy, how="outer", left_on='SRR_ID', right_on='run_accession', suffixes=[])


In [73]:
cols = all_df.columns.tolist()
cols_interest = [
  'BioProject', 'SRR_ID', 'title', 'library_strategy', 'library_name', 'fastq_ftp'
]

# find all columns that are missing from col_interest
cols_missing = list(set(cols) - set(cols_interest))

print(cols_missing)
# ['SRX_ID', 'Notes (searched using "Chlamydomonas" "metal" "toxicity" "SRA")',
# 'paper downloaded?', 'description', 'run_accession', 'fastq_bytes', 'DOI', 'library_source',
# 'fastq_md5', 'library_selection', 'Checked?', 'N_RUNS', 'included?', 'SRA_ID', 'Gene Expression Omnibus']

cols_filter_out = [
  'N_RUNS',
  'Checked?',
  'included?',
]
# remove the columns
cols_filtered = list(set(cols_missing) - set(cols_filter_out))
merge_cols = cols_interest + cols_filtered
print(merge_cols)
all_df = all_df[merge_cols]

['SRX_ID', 'Notes (searched using "Chlamydomonas" "metal" "toxicity" "SRA")', 'paper downloaded?', 'description', 'run_accession', 'fastq_bytes', 'DOI', 'library_source', 'fastq_md5', 'library_selection', 'Checked?', 'N_RUNS', 'included?', 'SRA_ID', 'Gene Expression Omnibus']
['BioProject', 'SRR_ID', 'title', 'library_strategy', 'library_name', 'fastq_ftp', 'SRX_ID', 'Notes (searched using "Chlamydomonas" "metal" "toxicity" "SRA")', 'paper downloaded?', 'description', 'run_accession', 'fastq_bytes', 'DOI', 'library_source', 'fastq_md5', 'library_selection', 'SRA_ID', 'Gene Expression Omnibus']


In [77]:
# change type to int
all_df['paper downloaded?'] = all_df['paper downloaded?'].astype(int)

# change type to bool
all_df['paper downloaded?'] = all_df['paper downloaded?'].astype(bool)

# order by BioProject, SRR_ID, then by 'library_name'
all_df = all_df.sort_values(['BioProject', 'SRR_ID', 'library_name'])

In [78]:
all_df.to_csv('../references/sra_details.csv', index=False)
all_df['SRR_ID'].to_csv('../references/srr_id_list.txt', header=None, index=None) # type: ignore
all_df[['BioProject', 'SRR_ID', 'title', 'library_strategy', 'library_name', 'fastq_ftp']].head()



,BioProject,SRR_ID,title,library_strategy,library_name,fastq_ftp
69,PRJNA335844,SRR3987296,GSM2255675: ElodeaValcea1; Elodea nuttallii; R...,RNA-Seq,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/SRR398/006/SRR398...
68,PRJNA335844,SRR3987297,GSM2255676: ElodeaValcea2; Elodea nuttallii; R...,RNA-Seq,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/SRR398/007/SRR398...
67,PRJNA335844,SRR3987298,GSM2255677: ElodeaValcea3; Elodea nuttallii; R...,RNA-Seq,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/SRR398/008/SRR398...
66,PRJNA335844,SRR3987299,GSM2255678: ElodeaBabeni1-1; Elodea nuttallii;...,RNA-Seq,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/SRR398/009/SRR398...
65,PRJNA335844,SRR3987300,GSM2255679: ElodeaBabeni1-2; Elodea nuttallii;...,RNA-Seq,NaN,ftp.sra.ebi.ac.uk/vol1/fastq/SRR398/000/SRR398...


In [79]:
# group by bioproject, then export the SRR_IDs as a txt list
bioprojects = all_df.groupby('BioProject')
for bioproject, group in bioprojects:
  group['fastq_ftp'].to_csv(f'../references/{bioproject}_srr_id_list.txt', header=None, index=None) # type: ignore

In [81]:
import pandas as pd
all_df = pd.read_csv('../references/sra_details.csv')

not_PRJNA576974 = all_df[all_df['BioProject'] != 'PRJNA576974']
not_PRJNA576974['fastq_ftp'].to_csv(f'../references/ftp_list.txt', header=None, index=None) # type: ignore